### 1. Imports and Utility Functions

This cell imports the required libraries and custom utility functions used throughout the notebook.  
The utilities handle image reading, segmentation, metric computation, visualization, and dataset path management.  
All core processing logic is encapsulated in `WB_utils.py` to keep the notebook clean and focused on experimentation.

In [1]:
import os
import cv2
import numpy as np

from WB_utils import (
    read_rgb,
    read_gray,
    segmentation_pipeline,
    compute_metrics,
    save_detailed_visuals,
    save_metrics_barchart,
    get_dataset_paths,
    ensure_mask_size_like_image
)


### 2. Dataset Processing and Performance Evaluation

This cell iterates over the dataset, applies the segmentation pipeline to each image, and compares the predicted masks with the ground truth masks.  
For each sample, standard segmentation metrics (Accuracy, Precision, Recall, F1-Score, and IoU) are computed.  
A subset of images is visualized for qualitative analysis, and the average performance metrics over the entire dataset are summarized and plotted.

In [2]:
image_paths, mask_paths = get_dataset_paths(verbose=True)

metrics = {
    "Accuracy": [],
    "Precision": [],
    "Recall": [],
    "F1-Score": [],
    "IoU": []
}

NUM_SAMPLE_IMAGES = 5

for i, (img_p, mask_p) in enumerate(zip(image_paths, mask_paths)):
    name = os.path.basename(img_p)

    img = read_rgb(img_p, verbose=True)
    gt  = read_gray(mask_p, verbose=True)

    if img is None or gt is None:
        continue

    gt = ensure_mask_size_like_image(gt, img)

    pred, s_eq, otsu_val = segmentation_pipeline(img)

    acc, prec, rec, f1, iou = compute_metrics(gt, pred)

    metrics["Accuracy"].append(acc)
    metrics["Precision"].append(prec)
    metrics["Recall"].append(rec)
    metrics["F1-Score"].append(f1)
    metrics["IoU"].append(iou)

    if i < NUM_SAMPLE_IMAGES:
        save_detailed_visuals(img, s_eq, gt, pred, otsu_val, name)

avg_metrics = {k: (float(np.mean(v)) if len(v) else 0.0) for k, v in metrics.items()}
save_metrics_barchart(avg_metrics)

avg_metrics


[INFO] Found 2841 images and 2841 masks.


{'Accuracy': 0.5596575103358492,
 'Precision': 0.38218908485917974,
 'Recall': 0.4984269101635385,
 'F1-Score': 0.3878490752239281,
 'IoU': 0.2958232829889976}